# 导入包

In [1]:
import random
import gymnasium as gym
import numpy as np
import torch
import torch.nn.functional as F
import rl_utils
import os

# 策略网络和Q网络

In [2]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)
        self.action_bound = action_bound  # action_bound是环境可以接受的动作最大值

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x)) * self.action_bound  # 缩放到动作空间


class QValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(QValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x, a):
        cat = torch.cat([x, a], dim=1)  # 拼接状态和动作
        x = F.relu(self.fc1(cat))
        x = F.relu(self.fc2(x))
        return self.fc_out(x)

# DDPG算法

In [3]:
class DDPG:
    ''' DDPG算法 '''

    def __init__(self, state_dim, hidden_dim, action_dim, action_bound,
                 sigma, actor_lr, critic_lr, tau, gamma, device):
        self.actor = PolicyNet(state_dim, hidden_dim,
                               action_dim, action_bound).to(device)
        self.critic = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        self.target_actor = PolicyNet(
            state_dim, hidden_dim, action_dim, action_bound).to(device)
        self.target_critic = QValueNet(
            state_dim, hidden_dim, action_dim).to(device)
        # 初始化目标价值网络并设置和价值网络相同的参数
        self.target_critic.load_state_dict(self.critic.state_dict())
        # 初始化目标策略网络并设置和策略相同的参数
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(
            self.actor.parameters(), lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(
            self.critic.parameters(), lr=critic_lr)
        self.gamma = gamma
        self.sigma = sigma  # 高斯噪声的标准差, 均值直接设为0
        self.tau = tau  # 目标网络软更新参数
        self.action_dim = action_dim
        self.device = device

    def take_action(self, state):
        state = torch.tensor(
            np.array(state), dtype=torch.float).to(self.device)
        action = self.actor(state).item()  # * 在这里和网络分离了, 因此后面不需要.item()
        # 给动作添加噪声，增加探索
        action = action + self.sigma * np.random.randn(self.action_dim)
        return action

    def soft_update(self, net, target_net):
        '''将target_net往net方向软更新, 每次更新幅度都很小

        参数说明
        ----------
        net : torch.nn.module
        target_net : torch.nn.module
        '''
        for param_target, param in zip(target_net.parameters(), net.parameters()):
            param_target.data.copy_(
                param_target.data * (1.0 - self.tau) + param.data * self.tau)

    def update(self, transition_dict):
        states = torch.tensor(
            transition_dict['states'], dtype=torch.float).to(self.device)
        actions = torch.tensor(
            transition_dict['actions'], dtype=torch.float).view(-1, 1).to(self.device)
        rewards = torch.tensor(
            transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(
            transition_dict['next_states'], dtype=torch.float).to(self.device)
        dones = torch.tensor(
            transition_dict['dones'], dtype=torch.int).view(-1, 1).to(self.device)
        truncated = torch.tensor(
            transition_dict['truncated'], dtype=torch.int).view(-1, 1).to(self.device)

        # 评论员还是时序差分更新, 但这次评论员鼠鼠变成Q网络了, 需要输入状态和动作, 动作由演员选择
        # 之前都是价值网络, 只需要输入状态
        next_q_values = self.target_critic(
            next_states, self.target_actor(next_states))
        q_targets = rewards + self.gamma * \
            next_q_values * (1 - (dones | truncated))
        critic_loss = torch.mean(F.mse_loss(
            self.critic(states, actions), q_targets))
        # 评论员梯度下降
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        # * 注意: 演员是梯度策略, 采用梯度上升, 加负号
        actor_loss = -torch.mean(self.critic(states, self.actor(states)))
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        # 一直更新, 但是缓慢更新
        self.soft_update(self.actor, self.target_actor)  # 软更新策略网络
        self.soft_update(self.critic, self.target_critic)  # 软更新价值网络


# 参数初始化

In [4]:
# DDPG算法相关
PATH = 'checkpoints/DDPG_PDv1.pt'
actor_lr = 3e-4
critic_lr = 3e-3
total_episodes = 200
total_epochs = 10
gamma = 0.98
tau = 0.005  # 软更新参数, tau越小更新幅度越小
buffer_size = 10000
minimal_size = 1000
batch_size = 64
sigma = 0.01  # 高斯噪声标准差
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
replay_buffer = rl_utils.ReplayBuffer(buffer_size)

# 环境相关
env_name = 'Pendulum-v1'
env = gym.make(env_name)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# 神经网络相关
state_dim = env.observation_space.shape[0]
hidden_dim = 64
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high[0]  # 动作最大值

agent = DDPG(state_dim, hidden_dim, action_dim, action_bound,
             sigma, actor_lr, critic_lr, tau, gamma, device)

# 检查点函数

In [11]:
def read_ckp(ckp_path):
    if os.path.exists(ckp_path):
        checkpoint = torch.load(ckp_path)
        s_epoch = checkpoint['epoch']
        s_episode = checkpoint['episode']
        agent.actor.load_state_dict(checkpoint['actor_best_weight'])
        agent.critic.load_state_dict(checkpoint['critic_best_weight'])
        return_list = checkpoint['return_list']
        return s_epoch, s_episode, return_list
    else:
        s_epoch = 0
        s_episode = 0
        return_list = []
        return s_epoch, s_episode, return_list

## 读取检查点

In [14]:
s_epoch, s_episode, return_list = read_ckp(PATH)

# 训练函数

训练速度变慢了, 可能是因为存在四个网络

In [15]:
return_list = rl_utils.train_off_policy_agent(
    env, agent, s_epoch, total_epochs, s_episode, total_episodes, replay_buffer, minimal_size, batch_size, return_list, PATH)

<6/10>:  97%|█████████▋| 194/200 [02:51<00:04,  1.24it/s, episode=1190, recent_return=-113.097]

# 作图

In [ ]:
rl_utils.picture_return(return_list, 'DDPG', env_name, 9)